Install required packages

In [1]:
!pip install transformers datasets soundfile accelerate speechbrain==0.5.16

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 38.5 MB/s eta 0:00:00


Login to Huggingface

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ug83nb5i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ug83nb5i
  Resolved https://github.com/huggingface/transformers.git to commit 6432ad8bb5dec9c7ece1041767c9e208ff6b4cbb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10035552 sha256=13747b63ee912e947b83ff968f8d7c5339f58853aa947cdccb30c7b93aa6448f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ppmf2agk/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      S

In [4]:
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


Do we have a GPU?

In [5]:
!nvidia-smi

Thu Oct 24 14:20:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#Load the model

In [6]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

#Load the Dataset

In [7]:
from datasets import load_dataset, Audio

dataset = load_dataset(
    "ymoslem/MediaSpeech", "fr", split="train"
)

README.md:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/320M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/321M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2498 [00:00<?, ? examples/s]

In [8]:
len(dataset)

2498

In [9]:
print(dataset)

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 2498
})


In [10]:
# Calculate the number of rows for half the dataset
half_size = len(dataset) // 16

# Select the first half of the dataset
dataset = dataset.select(range(half_size))

print(dataset)

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 156
})


It's important to set the sampling rate of the audio data to 16 kHz, which is what SpeechT5 expects.

In [11]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [12]:
tokenizer = processor.tokenizer

In [13]:
dataset[2:5]

{'audio': [{'path': 'b0a6f83a-9955-465d-afa7-b97ee04fda20.flac',
   'array': array([0., 0., 0., ..., 0., 0., 0.]),
   'sampling_rate': 16000},
  {'path': '977bf253-9777-40e2-b94a-a6c72dc60165.flac',
   'array': array([0., 0., 0., ..., 0., 0., 0.]),
   'sampling_rate': 16000},
  {'path': '06de4e33-9340-4c54-9404-d0b96d40bbf5.flac',
   'array': array([0., 0., 0., ..., 0., 0., 0.]),
   'sampling_rate': 16000}],
 'sentence': ["donc tout ça ce sont des mythes qui sont construites et puis il y a un personnage qui tranche pour moi avec l'ensemble c'est pierre laval qui est le qui a été président du conseil qui est un ancien socialiste et alors effectivement lui",
  "nourri logé blanchi on a des maris on a des enfants il faut payer le loyer et on en a marre d'être à découvert le vin du mois donc c'est plus possible et puis on veut aussi pouvoir bien s'occuper des patients parce que quand vous avez un patient qui va",
  "parce que nulle part ailleurs ne pense le politique c'estàdire expliquezvo

Let's normalize the dataset, create a column called "normalized_text"

In [16]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

In [17]:
dataset_vocab - tokenizer_vocab

{' ', 'à', 'â', 'ç', 'è', 'ô', 'ù', 'û'}

In [20]:
import re

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation (except apostrophes)
    text = re.sub(r'[^\w\s\']', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['sentence'] = normalize_text(example['sentence'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])


Map:   0%|          | 0/156 [00:00<?, ? examples/s]

{'audio': [{'path': 'b0a6f83a-9955-465d-afa7-b97ee04fda20.flac', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': '977bf253-9777-40e2-b94a-a6c72dc60165.flac', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': '06de4e33-9340-4c54-9404-d0b96d40bbf5.flac', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}], 'sentence': ["donc tout ça ce sont des mythes qui sont construites et puis il y a un personnage qui tranche pour moi avec l'ensemble c'est pierre laval qui est le qui a été président du conseil qui est un ancien socialiste et alors effectivement lui", "nourri logé blanchi on a des maris on a des enfants il faut payer le loyer et on en a marre d'être à découvert le vin du mois donc c'est plus possible et puis on veut aussi pouvoir bien s'occuper des patients parce que quand vous avez un patient qui va", "parce que nulle part ailleurs ne pense le politique c'estàdire expliquezvous il faut que je m'expliqu

In [21]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

In [22]:
dataset_vocab - tokenizer_vocab

{' ', 'à', 'â', 'ç', 'è', 'ô', 'ù', 'û'}

In [24]:
replacements = [
    ("â", "a"),  # Long a
    ("ç", "ch"),  # Ch as in "chair"
    ("ğ", "gh"),  # Silent g or slight elongation of the preceding vowel
    ("ı", "i"),   # Dotless i
    ("î", "i"),   # Long i
    ("ö", "oe"),  # Similar to German ö
    ("ş", "sh"),  # Sh as in "shoe"
    ("ü", "ue"),  # Similar to German ü
    ("û", "u"),   # Long u
]

def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["sentence"] = inputs["sentence"].replace(src, dst)
    return inputs

dataset = dataset.map(cleanup_text)

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

In [25]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(path, map_location=device), strict=Fal

In [30]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["sentence"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [31]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [32]:
processed_example["speaker_embeddings"].shape

(512,)

In [33]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

In [34]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

Filter:   0%|          | 0/156 [00:00<?, ? examples/s]

7

In [35]:
dataset = dataset.train_test_split(test_size=0.1)

In [36]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )# not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [37]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [39]:
from transformers import SpeechT5ForTextToSpeech

# Define or assign the checkpoint variable
checkpoint = "microsoft/speecht5_tts" # Replace with the correct checkpoint identifier or path

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

In [40]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)


In [41]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="TTS_FineTune_French",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=600,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [42]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

<ipython-input-42-3e9fd9c3f140>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


In [43]:
trainer.train()

Step,Training Loss,Validation Loss
100,1.076400,0.499496
200,0.870500,0.519676
300,0.786600,0.546931
400,0.706200,0.561532
500,0.671000,0.580451
600,0.618400,0.586972


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2816: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 1876}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=600, training_loss=0.8497156651814779, metrics={'train_runtime': 488.6129, 'train_samples_per_second': 39.295, 'train_steps_per_second': 1.228, 'total_flos': 601789809532032.0, 'train_loss': 0.8497156651814779, 'epoch': 600.0})

In [44]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/DeepDiveDev/TTS_FineTune_French/commit/1f81336f098354c853849675706f08847531ec89', commit_message='End of training', commit_description='', oid='1f81336f098354c853849675706f08847531ec89', pr_url=None, pr_revision=None, pr_num=None)

Inference

In [45]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "TTS_FineTune_French"
)

In [50]:
example = dataset["train"][5]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [51]:
text= "Salut ! Comment ça va ? Passe une excellente journée !"

In [52]:
number_words = {
    0: "zéro", 1: "un", 2: "deux", 3: "trois", 4: "quatre", 5: "cinq", 6: "six", 7: "sept", 8: "huit", 9: "neuf",
    10: "dix", 11: "onze", 12: "douze", 13: "treize", 14: "quatorze", 15: "quinze", 16: "seize", 17: "dix-sept",
    18: "dix-huit", 19: "dix-neuf", 20: "vingt", 30: "trente", 40: "quarante", 50: "cinquante", 60: "soixante",
    70: "soixante-dix", 80: "quatre-vingts", 90: "quatre-vingt-dix", 100: "cent", 1000: "mille"
}

def number_to_words(number):
    if number < 20:
        return number_words[number]
    elif number < 100:
        tens, unit = divmod(number, 10)
        return number_words[tens * 10] + (" " + number_words[unit] if unit else "")
    elif number < 1000:
        hundreds, remainder = divmod(number, 100)
        return (number_words[hundreds] + " cent" if hundreds > 1 else "cent") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000:
        thousands, remainder = divmod(number, 1000)
        return (number_to_words(thousands) + " mille" if thousands > 1 else "mille") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000:
        millions, remainder = divmod(number, 1000000)
        return number_to_words(millions) + " million" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000000:
        billions, remainder = divmod(number, 1000000000)
        return number_to_words(billions) + " milliard" + (" " + number_to_words(remainder) if remainder else "")
    else:
        return str(number)

def replace_numbers_with_words(text):
    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    # Find the numbers and replace them with words.
    result = re.sub(r'\b\d+\b', replace, text)

    return result

In [53]:
# Function to clean up text using the replacement pairs
def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [54]:
converted_text = replace_numbers_with_words(text)
cleaned_text = cleanup_text(converted_text)
final_text = normalize_text(cleaned_text)
final_text

'salut comment cha va passe une excellente journée'

In [55]:
inputs = processor(text=final_text, return_tensors="pt")

In [56]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

Generating audio

In [57]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
# Save the audio to a file (e.g., 'output.wav')
sf.write('output.wav', speech.numpy(), 16000)

In [58]:
!pip install transformers torch scipy soundfile

In [62]:
import torch
from transformers import AutoTokenizer, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from scipy.io.wavfile import write

# Load the fine-tuned model and tokenizer
model_name = "TTS_FineTune_French"  # Replace with your fine-tuned model path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = SpeechT5ForTextToSpeech.from_pretrained(model_name)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan") # load vocoder here

# Set the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
vocoder.to(device) # Move the vocoder to the same device as the model


def generate_audio(text, output_file, speaker_embeddings=None): # Add speaker_embeddings as an argument to the function with a default value of None
    # Tokenize input text
    inputs = tokenizer(text=text, return_tensors="pt").to(device) # sending inputs to the device

    # Generate audio features
    with torch.no_grad():
        # Check if speaker_embeddings is provided, otherwise use default values (e.g., random speaker embedding)
        if speaker_embeddings is None:
            # Load or create default speaker embeddings here if needed
            # For demonstration, using a random embedding of size (1, 512)
            speaker_embeddings = torch.randn(1, 512).to(device)  # Adjust size if necessary

        # Generate audio features using model.generate_speech
        speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    # Convert audio features to waveform
    waveform = speech.cpu().numpy()

    # Write the waveform to a WAV file
    write(output_file, 16000, waveform)  # Using a sample rate of 16000 which is the typical sample rate for SpeechT5

# Example usage
text_input = "Bonjour, comment ça va? Passez une excellente journée!"  # Replace with your desired text
output_audio_file = "output_audio.wav"

# Assuming you have speaker embeddings from the previous part of your code, pass it here
# If not, generate_audio will use the default/random embeddings
generate_audio(text_input, output_audio_file, speaker_embeddings=None)
print(f"Audio generated and saved as {output_audio_file}.")

Audio generated and saved as output_audio.wav.


In [68]:
!pip install pygobject
import os

# Check if the file exists
file_path = '/content/output_audio.wav'
if os.path.exists(file_path):
    print(f"File exists at: {file_path}")
else:
    print(f"File not found at: {file_path}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.5/661.5 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
File exists at: /content/output_audio.wav


In [69]:

!pip install bark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 7.4 MB/s eta 0:00:00
  Created wheel for encodec: filename=encodec-0.1.1-py3-none-any.whl size=45760 sha256=76c03f16720547a51ea14017e552634d152bb275c2cad2c13a17d311674bbaa2
  Stored in directory: /root/.cache/pip/wheels/fc/36/cb/81af8b985a5f5e0815312d5e52b41263237af07b977e6bcbf3
Successfully built encodec


In [70]:
from bark import generate_audio

# Your input text in French
text = "Salut! Comment ça va? Passe une excellente journée."

# Generate audio
audio_array = generate_audio(text)

# Save the generated audio
import soundfile as sf
sf.write('output_audio.wav', audio_array, 24000)  # Assuming Bark uses a sample rate of 24000 Hz

# Play the audio (for Jupyter notebooks)
from IPython.display import Audio
Audio('output_audio.wav')

text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bark/generation.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 161MB/s]
/usr/local/lib/python3.10/dist-packages/bark/generation.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with InferenceContext(), torch.inference_mode(), torch.no_grad(), autocast():
100%|██████████| 14/14 [00:17<00:00,  1.28s/it]


In [71]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [72]:
!git config --global user.name "Jyotirmoyee18"
!git config --global user.email "Jyotirmoyeemandal63@gmail.com"

In [75]:
%cd https://github.com/Jyotirmoyee18/TTS_FineTuning_French.git

[Errno 2] No such file or directory: 'https://github.com/Jyotirmoyee18/TTS_FineTuning_French.git'
/content


In [76]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [77]:
!cp /content/TTS_FineTune_French* /content/TTS_FineTune_French

cp: -r not specified; omitting directory '/content/TTS_FineTune_French'


In [78]:
!git add .
!git commit -m "Add TTS_FineTune_French"

^C
On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	TTS_FineTune_French/
	output.wav
	output_audio.wav
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


In [79]:
!zip -r TTS_FineTune_French.zip /content/TTS_FineTune_French

  adding: content/TTS_FineTune_French/ (stored 0%)
  adding: content/TTS_FineTune_French/checkpoint-600/ (stored 0%)
  adding: content/TTS_FineTune_French/checkpoint-600/added_tokens.json (deflated 13%)
  adding: content/TTS_FineTune_French/checkpoint-600/scheduler.pt (deflated 55%)
  adding: content/TTS_FineTune_French/checkpoint-600/optimizer.pt


zip error: Interrupted (aborting)


In [81]:
from google.colab import files
files.download('/content/TTS_FineTune_French')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>